In [26]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [124]:
import pandas as pd
import re
import os

def filter_data(input_file_path, output_file_path,keyword):
    for file in os.listdir(input_file_path):
        dataFrame= pd.read_csv(os.path.join(input_file_path, file))
        # select rows containing text "Affirm"
        data_filtered = dataFrame.loc[dataFrame['DESCRIPTION'].str.contains(keyword),['DATE', 'DESCRIPTION', 'RECEIVED']]
        # data_filtered.to_csv(os.path.join(output_file_path, file), index = False)

        #split the decription
        new =data_filtered.DESCRIPTION.str.split(expand=True)
        data_filtered['DEPOSIT_ID']=new[15]
        data_filtered.drop(columns =["DESCRIPTION"], inplace = True)
        data_filtered['DESCRIPTION']=new[3]+' '+new[4]

        # split the deposit id
        new_id=data_filtered.DEPOSIT_ID.str.split(":",expand=True)
        data_filtered.drop(columns =["DEPOSIT_ID"], inplace = True)
        data_filtered['DEPOSIT_ID']=new_id[1]
        
        print(data_filtered)
        
    

In [ ]:
filter_data('/content/drive/MyDrive/Colab Notebooks/input_file_path','/content/drive/MyDrive/Colab Notebooks/output_path','AFFIRM')
